In [1]:
import pandas as pd
import json
from functools import reduce

In [10]:
from import_helm_lite import read_helm_list

In [2]:
runs = '/nlp/scr4/nlp/crfm/yifanmai/helm-release/benchmark_output/runs/'
version = 'v1.11.0/'
instance = 'natural_qa:mode=closedbook,model=meta_llama-3.3-70b-instruct-turbo/'
path = runs + version + instance


In [3]:
with open(path + 'display_requests.json') as json_file:
    d = json.load(json_file)

In [4]:
relevant_files = {'display_predictions': ['instance_id', 'predicted_text', 'stats'],
 # 'display_requests': ['instance_id', 'prompt'],
 'instances': ['id', 'input', 'references']}

In [5]:
# pd.json_normalize(test)

In [6]:
df_list = []
for file in relevant_files.keys():
    with open(path + file + '.json') as json_file:
        full_dict = json.load(json_file)
    
    only_relevant_keys = [{key: record[key] for key in relevant_files[file]} for record in full_dict]

    df_temp = pd.json_normalize(only_relevant_keys)
    if file=='instances':
        df_temp = df_temp.rename(columns={'id': 'instance_id'})
    df_list.append(df_temp)

In [7]:
df = reduce(
    lambda left, right: pd.merge(left, right, on="instance_id", how="outer"),
    df_list
)

In [8]:
df

,instance_id,predicted_text,stats.num_prompt_tokens,stats.num_output_tokens,stats.inference_runtime,stats.num_train_instances,stats.prompt_truncated,stats.f1_score,references,input.text
0,id1002,Tungabhadra,129.0,6.0,0.303247,5.0,0.0,0.000000,"[{'output': {'text': 'Tandi, in Lahaul'}, 'tag...",Chandra and bhaga river meets at the place?
1,id1004,1947,127.0,3.0,0.474435,5.0,0.0,0.500000,[{'output': {'text': 'during a meeting of the ...,When was the national flag of india adopted?
2,id1008,"Bob Dylan, George Harrison, Jeff Lynne, Roy Or...",129.0,19.0,0.322746,5.0,0.0,0.307692,"[{'output': {'text': 'Bob Dylan'}, 'tags': ['c...",Who are the artists in the traveling wilburys?
3,id1012,Reproductive organs (sperm and egg cells),129.0,11.0,0.348055,5.0,0.0,0.600000,"[{'output': {'text': 'sperm'}, 'tags': ['corre...",Where are haploid cells found in the human body?
4,id1014,"Various locations, but primarily in the United...",129.0,24.0,0.749017,5.0,0.0,0.000000,"[{'output': {'text': 'Hotel Pennsylvania, New ...",Where is the opposition with jordan klepper fi...
...,...,...,...,...,...,...,...,...,...,...
995,id974,Rump,128.0,3.0,0.388989,5.0,0.0,0.000000,[{'output': {'text': 'the rear leg of the cow'...,Where does eye of the round roast come from?
996,id976,5,127.0,2.0,0.508516,5.0,0.0,1.000000,"[{'output': {'text': 'five'}, 'tags': ['correc...",How many seasons are there for lost girl?
997,id978,"Victoria, Canada (then part of the Dominion of...",128.0,123.0,1.757440,5.0,0.0,0.062500,"[{'output': {'text': 'Montreal'}, 'tags': ['co...",Where was the first nhl hockey game played?
998,id980,Inside and outside the cell.,127.0,7.0,0.356390,5.0,0.0,0.000000,"[{'output': {'text': 'from other cells'}, 'tag...",Where do the signals for apoptosis come from?


In [9]:
df['references'].iloc[0:5].values

array([list([{'output': {'text': 'Tandi, in Lahaul'}, 'tags': ['correct']}, {'output': {'text': 'at Tandi, in Lahaul'}, 'tags': ['correct']}]),
       list([{'output': {'text': 'during a meeting of the Constituent Assembly held on 22 July 1947'}, 'tags': ['correct']}, {'output': {'text': '22 July 1947'}, 'tags': ['correct']}, {'output': {'text': '15 August 1947'}, 'tags': ['correct']}]),
       list([{'output': {'text': 'Bob Dylan'}, 'tags': ['correct']}, {'output': {'text': 'George Harrison'}, 'tags': ['correct']}, {'output': {'text': 'Jeff Lynne'}, 'tags': ['correct']}, {'output': {'text': 'Roy Orbison'}, 'tags': ['correct']}, {'output': {'text': 'Tom Petty'}, 'tags': ['correct']}]),
       list([{'output': {'text': 'sperm'}, 'tags': ['correct']}, {'output': {'text': 'ova'}, 'tags': ['correct']}, {'output': {'text': 'sperm or egg cells'}, 'tags': ['correct']}, {'output': {'text': 'gametes (sperm or egg cells)'}, 'tags': ['correct']}]),
       list([{'output': {'text': 'Hotel Pennsylv

In [12]:
df['all_tags_correct'] = df['references'].apply(
    lambda refs: all(tag == 'correct' for ref in refs for tag in ref['tags'])
)

In [14]:
df['all_tags_correct'].sum()

np.int64(1000)